In [ ]:
!sudo apt-get install libportaudio2
!pip install sounddevice
!pip install tflite-model-maker
!pip install seedir

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 1s (106 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 6.6 MB/s 
     |████████████████████████████████| 175 kB 40.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=9351911a24d1dfa978f898a34337d45abd6b55812399d93643ac21df7ac04383
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader
import sounddevice as sd
import seedir as sed
import numpy as np
import pandas as pd
from pathlib import Path
import os
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import layers

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
image_path = '/content/drive/My Drive/'
image_path = os.path.join(os.path.dirname(image_path), 'eks2/')

In [ ]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.8)
#test_data, val_data = rest_data.split(0.5)

INFO:tensorflow:Load image with size: 3908, num_label: 10, labels: ayam bakar, bakso, bandeng presto, gado gado, gudeg, lumpia semarang, nasi pindang, rendang, sate, telur_asin.


INFO:tensorflow:Load image with size: 3908, num_label: 10, labels: ayam bakar, bakso, bandeng presto, gado gado, gudeg, lumpia semarang, nasi pindang, rendang, sate, telur_asin.


In [ ]:
efficientnet_lite0=image_classifier.EfficientNetLite0Spec(
    uri='https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2'
)

In [ ]:
from tensorflow.keras import layers

data_augment = tf.keras.Sequential([
  layers.Rescaling(1./255),
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.2),
  layers.RandomZoom(0.4)
])

In [ ]:
#model0 = image_classifier.create(train_data, model_spec=inception_v3_spec, validation_data=val_data)
model1 = image_classifier.create(train_data, model_spec=efficientnet_lite0, 
                                 validation_data=test_data,batch_size=32,
                                 epochs=15,dropout_rate=0.7,
                                 learning_rate=0.001,train_whole_model=True)

    

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_1 (Hub  (None, 1280)             3413024   
 KerasLayerV1V2)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 10)                12810     
                                                                 
Total params: 3,425,834
Trainable params: 3,425,834
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


97/97 [==============================] - 78s 772ms/step - loss: 1.9495 - accuracy: 0.3557 - val_loss: 1.2982 - val_accuracy: 0.7187
Epoch 2/15
97/97 [==============================] - 88s 915ms/step - loss: 1.2979 - accuracy: 0.6746 - val_loss: 0.9927 - val_accuracy: 0.8159
Epoch 3/15
97/97 [==============================] - 68s 704ms/step - loss: 1.0882 - accuracy: 0.7761 - val_loss: 0.8624 - val_accuracy: 0.8683
Epoch 4/15
97/97 [==============================] - 88s 913ms/step - loss: 0.9860 - accuracy: 0.8283 - val_loss: 0.8361 - val_accuracy: 0.8772
Epoch 5/15
97/97 [==============================] - 71s 736ms/step - loss: 0.8920 - accuracy: 0.8776 - val_loss: 0.8341 - val_accuracy: 0.8760
Epoch 6/15
97/97 [==============================] - 88s 911ms/step - loss: 0.8546 - accuracy: 0.8914 - val_loss: 0.8259 - val_accuracy: 0.8836
Epoch 7/15
97/97 [==============================] - 88s 913ms/step - loss: 0.8298 - accuracy: 0.8947 - val_loss: 0.8165 - val_accuracy: 0.8875
Epoch 8/15

In [ ]:
config = QuantizationConfig.for_dynamic()
model1.export(export_dir='.', tflite_filename='foodictive_dynamic_quantize.tflite', quantization_config=config)
model1.export(export_dir='.', export_format=ExportFormat.LABEL)

INFO:tensorflow:Assets written to: /tmp/tmp0ry5mdzy/assets


INFO:tensorflow:Assets written to: /tmp/tmp0ry5mdzy/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp117nppaf/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmp117nppaf/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./foodictive_dynamic_quantize.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./foodictive_dynamic_quantize.tflite


INFO:tensorflow:Saving labels in ./labels.txt


INFO:tensorflow:Saving labels in ./labels.txt
